# 05 - Associations et Interfaces

⚡ Intermédiaire | ⏱ 45 min | 🔑 Concepts : composition, agrégation, interfaces, contrats

## Objectifs

- Comprendre les différents types d'associations entre classes
- Maîtriser la composition et l'agrégation
- Savoir quand préférer la composition à l'héritage
- Comprendre le concept d'interface et de contrat
- Différencier classes abstraites et interfaces
- Découvrir le principe de responsabilité unique (SRP)
- Lire et créer des diagrammes UML d'associations

## Prérequis

- Comprendre l'héritage et le polymorphisme
- Connaître la notation UML de base
- Avoir des notions d'encapsulation

## 1. Association : Relation Entre Classes

### Définition

Une **association** est une relation sémantique entre deux classes qui permet aux objets d'une classe d'interagir avec les objets d'une autre classe.

### Types d'Associations

1. **Association simple** : Relation générale entre classes
2. **Agrégation** : Relation "a-un" faible (has-a weak)
3. **Composition** : Relation "a-un" forte (has-a strong)

### Notation UML

```
Association simple : ─────────
Agrégation         : ◇────────  (losange vide)
Composition        : ◆────────  (losange plein)
Héritage           : ──────▷   (flèche vide)
```

In [ ]:
# Exemple d'association simple

class Professeur:
    """Un professeur enseigne à des étudiants."""
    
    def __init__(self, nom, matiere):
        self.nom = nom
        self.matiere = matiere
        self.etudiants = []  # Association avec Etudiant
    
    def ajouter_etudiant(self, etudiant):
        self.etudiants.append(etudiant)
    
    def lister_etudiants(self):
        return [e.nom for e in self.etudiants]


class Etudiant:
    """Un étudiant suit des cours."""
    
    def __init__(self, nom, numero):
        self.nom = nom
        self.numero = numero


# Démonstration
prof = Professeur("Dr. Martin", "Python")
alice = Etudiant("Alice", "E001")
bob = Etudiant("Bob", "E002")

prof.ajouter_etudiant(alice)
prof.ajouter_etudiant(bob)

print(f"{prof.nom} enseigne {prof.matiere} à:")
for nom in prof.lister_etudiants():
    print(f"  - {nom}")

## 2. Composition : Relation "A-Un" Forte

### Définition

La **composition** est une relation où :
- Un objet **contient** d'autres objets
- Les objets contenus **n'existent pas** sans le conteneur
- Le cycle de vie des parties dépend du tout

### Caractéristiques

- **Dépendance forte** : Si le conteneur est détruit, les parties le sont aussi
- **Ownership** : Le conteneur "possède" les parties
- **Lifetime dependency** : Les parties naissent et meurent avec le conteneur

### Exemples

- Une Maison **contient** des Pièces (sans maison, les pièces n'existent pas)
- Une Voiture **contient** un Moteur (moteur créé avec la voiture)
- Un Livre **contient** des Chapitres

In [ ]:
# Exemple de composition forte

class Moteur:
    """Moteur d'une voiture."""
    
    def __init__(self, puissance):
        self.puissance = puissance
        self.est_demarre = False
    
    def demarrer(self):
        self.est_demarre = True
        return f"Moteur {self.puissance}ch démarré"
    
    def arreter(self):
        self.est_demarre = False
        return "Moteur arrêté"


class Voiture:
    """Voiture qui possède un moteur (composition)."""
    
    def __init__(self, marque, puissance_moteur):
        self.marque = marque
        # Le moteur est CREE par la voiture (composition)
        self.moteur = Moteur(puissance_moteur)
    
    def demarrer(self):
        return f"{self.marque}: {self.moteur.demarrer()}"
    
    def arreter(self):
        return f"{self.marque}: {self.moteur.arreter()}"
    
    def __del__(self):
        """Quand la voiture est détruite, le moteur l'est aussi."""
        print(f"La voiture {self.marque} est détruite (moteur inclus)")


# Démonstration
ma_voiture = Voiture("Tesla", 450)
print(ma_voiture.demarrer())
print(ma_voiture.arreter())

# Le moteur n'existe pas indépendamment de la voiture
# On ne peut pas faire : moteur = Moteur(450); voiture = Voiture("Tesla", moteur)

## 3. Agrégation : Relation "A-Un" Faible

### Définition

L'**agrégation** est une relation où :
- Un objet **contient** d'autres objets
- Les objets contenus **peuvent exister** indépendamment du conteneur
- Pas de dépendance de cycle de vie

### Caractéristiques

- **Dépendance faible** : Les parties survivent à la destruction du conteneur
- **Shared ownership** : Les parties peuvent appartenir à plusieurs conteneurs
- **Lifetime independence** : Les parties ont leur propre cycle de vie

### Exemples

- Une Équipe **a** des Joueurs (joueurs existent sans l'équipe)
- Une Bibliothèque **a** des Livres (livres existent indépendamment)
- Un Département **a** des Employés

In [ ]:
# Exemple d'agrégation faible

class Joueur:
    """Joueur qui peut appartenir à une équipe."""
    
    def __init__(self, nom, numero):
        self.nom = nom
        self.numero = numero
    
    def __repr__(self):
        return f"Joueur({self.nom}, #{self.numero})"


class Equipe:
    """Équipe qui regroupe des joueurs (agrégation)."""
    
    def __init__(self, nom):
        self.nom = nom
        self.joueurs = []  # Liste de joueurs existants
    
    def ajouter_joueur(self, joueur):
        """Ajoute un joueur EXISTANT à l'équipe."""
        self.joueurs.append(joueur)
    
    def retirer_joueur(self, joueur):
        """Retire un joueur (il continue d'exister)."""
        self.joueurs.remove(joueur)
    
    def __del__(self):
        print(f"L'équipe {self.nom} est dissoute (joueurs toujours existants)")


# Démonstration
# Les joueurs sont créés INDEPENDEMMENT de l'équipe
messi = Joueur("Messi", 10)
ronaldo = Joueur("Ronaldo", 7)
neymar = Joueur("Neymar", 11)

# L'équipe regroupe des joueurs existants
equipe_a = Equipe("Team A")
equipe_a.ajouter_joueur(messi)
equipe_a.ajouter_joueur(ronaldo)

print(f"Équipe {equipe_a.nom}: {equipe_a.joueurs}")

# Un joueur peut changer d'équipe
equipe_b = Equipe("Team B")
equipe_a.retirer_joueur(messi)
equipe_b.ajouter_joueur(messi)

print(f"\nAprès transfert:")
print(f"Team A: {equipe_a.joueurs}")
print(f"Team B: {equipe_b.joueurs}")

# Le joueur existe toujours même si l'équipe est supprimée
del equipe_b
print(f"\nMessi existe toujours: {messi}")

## 4. Composition vs Héritage : "Prefer Composition Over Inheritance"

### Le Principe

C'est un des principes fondamentaux du design orienté objet :

> **"Favorisez la composition plutôt que l'héritage"**

### Pourquoi ?

#### Problèmes de l'Héritage

1. **Couplage fort** : La classe fille dépend de l'implémentation de la classe mère
2. **Fragile** : Un changement dans la classe mère peut casser les filles
3. **Hiérarchie rigide** : Difficile de réorganiser
4. **Héritage multiple** : Problème du diamant

#### Avantages de la Composition

1. **Couplage faible** : Les objets communiquent via des interfaces
2. **Flexibilité** : On peut changer les composants à l'exécution
3. **Réutilisabilité** : Les composants sont indépendants
4. **Testabilité** : Plus facile de mocker les dépendances

### Quand Utiliser Quoi ?

| Situation | Utiliser |
|-----------|----------|
| Relation "est-un" claire | Héritage |
| Relation "a-un" | Composition |
| Comportement partagé | Composition |
| Polymorphisme nécessaire | Héritage ou Interfaces |
| Flexibilité runtime | Composition |

In [ ]:
# Comparaison Héritage vs Composition

# ❌ Mauvais : Héritage pour réutiliser du code
class Moteur:
    def demarrer(self):
        return "Moteur démarré"

class VoitureHeritee(Moteur):  # ❌ Une voiture N'EST PAS un moteur!
    pass


# ✅ Bon : Composition
class Moteur:
    def demarrer(self):
        return "Moteur démarré"

class VoitureComposee:  # ✅ Une voiture A un moteur
    def __init__(self):
        self.moteur = Moteur()
    
    def demarrer(self):
        return self.moteur.demarrer()


# Exemple plus complexe : système de notification

# ❌ Héritage : rigide
class NotificationEmail:
    def envoyer(self, message):
        return f"Email: {message}"

class UtilisateurAvecEmail(NotificationEmail):  # Rigide!
    def __init__(self, nom):
        self.nom = nom


# ✅ Composition : flexible
class NotificationEmail:
    def envoyer(self, message):
        return f"📧 Email: {message}"

class NotificationSMS:
    def envoyer(self, message):
        return f"📱 SMS: {message}"

class NotificationPush:
    def envoyer(self, message):
        return f"🔔 Push: {message}"

class Utilisateur:
    """Utilisateur avec notification configurable (composition)."""
    
    def __init__(self, nom, notification):
        self.nom = nom
        self.notification = notification  # Composition!
    
    def notifier(self, message):
        return self.notification.envoyer(f"{self.nom}: {message}")


# Flexibilité de la composition
alice = Utilisateur("Alice", NotificationEmail())
bob = Utilisateur("Bob", NotificationSMS())
charlie = Utilisateur("Charlie", NotificationPush())

print(alice.notifier("Nouveau message"))
print(bob.notifier("Nouveau message"))
print(charlie.notifier("Nouveau message"))

# On peut même changer la notification à l'exécution!
alice.notification = NotificationSMS()
print("\nAlice change de notification:")
print(alice.notifier("Encore un message"))

## 5. Interfaces : Contrat de Comportement

### Définition

Une **interface** est un contrat qui définit :
- **Quelles** méthodes une classe doit implémenter
- **Pas comment** ces méthodes sont implémentées

### Principe

"Je me fiche de qui tu es, tant que tu peux faire X, Y et Z."

### Avantages

1. **Contrat clair** : On sait exactement ce qu'une classe doit faire
2. **Polymorphisme** : Différentes classes peuvent implémenter la même interface
3. **Découplage** : Le code dépend d'abstractions, pas de concrétions
4. **Testabilité** : Facile de créer des mocks

### En Python

Python n'a pas de mot-clé `interface` comme Java ou C#. On utilise :
1. **Classes abstraites** (module `abc`)
2. **Duck typing** : "Si ça marche comme un canard..."
3. **Protocols** (Python 3.8+, typing)

In [ ]:
# Exemple d'interface avec ABC (Abstract Base Class)

from abc import ABC, abstractmethod


class Payable(ABC):
    """Interface : tout objet payable doit implémenter calculer_paiement()."""
    
    @abstractmethod
    def calculer_paiement(self) -> float:
        """Calcule le montant à payer."""
        pass


class Employe(Payable):
    """Un employé implémente l'interface Payable."""
    
    def __init__(self, nom, salaire_mensuel):
        self.nom = nom
        self.salaire_mensuel = salaire_mensuel
    
    def calculer_paiement(self) -> float:
        return self.salaire_mensuel


class Freelance(Payable):
    """Un freelance implémente aussi l'interface Payable."""
    
    def __init__(self, nom, taux_horaire, heures_travaillees):
        self.nom = nom
        self.taux_horaire = taux_horaire
        self.heures_travaillees = heures_travaillees
    
    def calculer_paiement(self) -> float:
        return self.taux_horaire * self.heures_travaillees


class Facture(Payable):
    """Une facture implémente aussi Payable."""
    
    def __init__(self, numero, montant):
        self.numero = numero
        self.montant = montant
    
    def calculer_paiement(self) -> float:
        return self.montant


# Fonction polymorphe qui accepte n'importe quel Payable
def traiter_paiement(payable: Payable):
    """Traite le paiement de n'importe quel objet Payable."""
    montant = payable.calculer_paiement()
    print(f"Paiement traité: {montant:.2f} €")
    return montant


# Démonstration
payables = [
    Employe("Alice", 3000),
    Freelance("Bob", 50, 40),
    Facture("F001", 1500)
]

total = 0
for p in payables:
    total += traiter_paiement(p)

print(f"\nTotal des paiements: {total:.2f} €")

## 6. Classes Abstraites vs Interfaces

### Classe Abstraite

- Peut avoir des méthodes **abstraites** (sans implémentation)
- Peut avoir des méthodes **concrètes** (avec implémentation)
- Peut avoir des **attributs**
- Ne peut pas être instanciée
- Utilisée pour **partager du code** entre classes liées

### Interface Pure

- Uniquement des méthodes **abstraites**
- Pas d'implémentation
- Pas d'attributs (ou seulement constantes)
- Définit un **contrat** pur
- Utilisée pour le **polymorphisme**

### En Python

Python n'a pas de distinction stricte. On utilise `ABC` pour les deux, mais on peut choisir le style :
- **Style interface** : Toutes les méthodes sont abstraites
- **Style classe abstraite** : Mix de méthodes abstraites et concrètes

In [ ]:
# Comparaison classe abstraite vs interface

from abc import ABC, abstractmethod


# Interface pure : seulement des méthodes abstraites
class Drawable(ABC):
    """Interface pure : contrat pour objets dessinables."""
    
    @abstractmethod
    def dessiner(self):
        """Dessine l'objet."""
        pass
    
    @abstractmethod
    def effacer(self):
        """Efface l'objet."""
        pass


# Classe abstraite : méthodes abstraites + code partagé
class Forme(ABC):
    """Classe abstraite avec comportement partagé."""
    
    def __init__(self, couleur):
        self.couleur = couleur
    
    @abstractmethod
    def aire(self):
        """Méthode abstraite : à implémenter."""
        pass
    
    # Méthode concrète : partagée par toutes les formes
    def description(self):
        return f"Forme {self.couleur} avec aire {self.aire():.2f}"


# Implémentation
class Cercle(Forme, Drawable):
    """Implémente Forme et Drawable."""
    
    def __init__(self, couleur, rayon):
        super().__init__(couleur)
        self.rayon = rayon
    
    def aire(self):
        import math
        return math.pi * self.rayon ** 2
    
    def dessiner(self):
        return f"Dessin d'un cercle {self.couleur} de rayon {self.rayon}"
    
    def effacer(self):
        return f"Effacement du cercle"


# Test
c = Cercle("rouge", 5)
print(c.dessiner())  # De Drawable
print(c.description())  # De Forme (méthode concrète)
print(c.effacer())  # De Drawable

# Impossible d'instancier une classe abstraite
try:
    f = Forme("bleu")  # ❌ Erreur!
except TypeError as e:
    print(f"\nErreur attendue: {e}")

## 7. Principe de Responsabilité Unique (SRP)

### Définition

Le **Single Responsibility Principle** (SRP) est le premier principe SOLID :

> **"Une classe ne devrait avoir qu'une seule raison de changer."**

### Interprétation

- Une classe = une **responsabilité**
- Une classe = un **rôle** dans le système
- Si une classe fait trop de choses → la diviser

### Avantages

1. **Cohésion** : Code logiquement groupé
2. **Maintenabilité** : Changements localisés
3. **Testabilité** : Tests plus simples
4. **Réutilisabilité** : Classes plus modulaires

### Exemple Classique

Une classe `Utilisateur` ne devrait pas :
- Gérer les données utilisateur
- Gérer l'authentification
- Envoyer des emails
- Logger les actions
- Sauvegarder en base de données

→ Trop de responsabilités! Diviser en plusieurs classes.

In [ ]:
# Violation du SRP

# ❌ Mauvais : trop de responsabilités
class UtilisateurV1:
    """Classe qui fait TOUT → viole SRP."""
    
    def __init__(self, nom, email, mot_de_passe):
        self.nom = nom
        self.email = email
        self.mot_de_passe = mot_de_passe
    
    # Responsabilité 1 : Authentification
    def authentifier(self, mdp):
        return self.mot_de_passe == mdp
    
    # Responsabilité 2 : Envoi d'emails
    def envoyer_email(self, message):
        return f"Email envoyé à {self.email}: {message}"
    
    # Responsabilité 3 : Sauvegarde en base
    def sauvegarder(self):
        return f"Utilisateur {self.nom} sauvegardé en DB"
    
    # Responsabilité 4 : Logging
    def logger_action(self, action):
        return f"[LOG] {self.nom}: {action}"

# Problème : cette classe change si on modifie n'importe lequel de ces aspects!


# ✅ Bon : respect du SRP

class Utilisateur:
    """Responsabilité : Données utilisateur seulement."""
    
    def __init__(self, nom, email):
        self.nom = nom
        self.email = email


class AuthentificationService:
    """Responsabilité : Authentification."""
    
    def __init__(self):
        self._mots_de_passe = {}  # Simulé
    
    def authentifier(self, utilisateur, mot_de_passe):
        mdp_stocke = self._mots_de_passe.get(utilisateur.email)
        return mdp_stocke == mot_de_passe


class EmailService:
    """Responsabilité : Envoi d'emails."""
    
    def envoyer(self, destinataire, message):
        return f"Email envoyé à {destinataire}: {message}"


class UtilisateurRepository:
    """Responsabilité : Sauvegarde des utilisateurs."""
    
    def sauvegarder(self, utilisateur):
        return f"Utilisateur {utilisateur.nom} sauvegardé en DB"


class Logger:
    """Responsabilité : Logging."""
    
    def log(self, message):
        return f"[LOG] {message}"


# Utilisation avec composition
user = Utilisateur("Alice", "alice@example.com")
auth_service = AuthentificationService()
email_service = EmailService()
user_repo = UtilisateurRepository()
logger = Logger()

# Chaque service a une responsabilité unique
print(user_repo.sauvegarder(user))
print(email_service.envoyer(user.email, "Bienvenue!"))
print(logger.log(f"{user.nom} s'est inscrit"))

## 8. Notation UML : Associations

### Types de Relations UML

```
Association simple:    ClasseA ────────── ClasseB
Agrégation:           ClasseA ◇────────── ClasseB
Composition:          ClasseA ◆────────── ClasseB
Héritage:             ClasseA ──────▷ ClasseB
Réalisation (interface): ClasseA ┄┄┄▷ InterfaceB
```

### Multiplicités

```
1      : exactement 1
0..1   : 0 ou 1
1..*   : 1 ou plus
0..*   : 0 ou plus (aussi noté *)
3..5   : entre 3 et 5
```

### Exemple Complet

```
Entreprise ◆──────── 1..* Département
Département ◇──────── 1..* Employé
```

Signification :
- Une Entreprise **contient** (composition) 1 ou plusieurs Départements
- Un Département **a** (agrégation) 1 ou plusieurs Employés

In [ ]:
# Diagramme Mermaid pour les associations

mermaid_code = """
classDiagram
    class Entreprise {
        +String nom
        +List~Departement~ departements
        +ajouterDepartement()
    }
    
    class Departement {
        +String nom
        +List~Employe~ employes
        +ajouterEmploye()
    }
    
    class Employe {
        +String nom
        +float salaire
        +travailler()
    }
    
    %% Composition : Entreprise possède Departement
    Entreprise *-- "1..*" Departement : contient
    
    %% Agrégation : Departement a des Employes
    Departement o-- "1..*" Employe : a
"""

print("Code Mermaid pour diagramme d'associations:")
print(mermaid_code)
print("\n📝 Visualisez sur https://mermaid.live")
print("\n🔑 Symboles:")
print("  *-- : Composition (losange plein)")
print("  o-- : Agrégation (losange vide)")
print("  <|-- : Héritage")
print("  <|.. : Réalisation (interface)")

## 9. Pièges Courants

### Piège 1 : Trop d'Héritage

**Problème** : Utiliser l'héritage par défaut au lieu de la composition.

In [ ]:
# Trop d'héritage

# ❌ Mauvais
class ListeAvecLog(list):  # Héritage de list
    def append(self, item):
        print(f"Ajout de {item}")
        super().append(item)

# Problème : on hérite de TOUTES les méthodes de list
# Que fait extend()? insert()? On doit tout surcharger!


# ✅ Meilleur : Composition
class ListeAvecLog:
    def __init__(self):
        self._liste = []  # Composition
    
    def append(self, item):
        print(f"Ajout de {item}")
        self._liste.append(item)
    
    def get_items(self):
        return self._liste.copy()

# Plus de contrôle, pas d'exposition de méthodes non voulues

### Piège 2 : God Class (Classe Dieu)

**Problème** : Une classe qui fait tout (violation massive du SRP).

In [ ]:
# God Class

# ❌ Classe Dieu : fait TOUT
class SystemeGestion:
    """Classe qui gère tout dans l'application."""
    
    def creer_utilisateur(self): pass
    def supprimer_utilisateur(self): pass
    def envoyer_email(self): pass
    def generer_rapport(self): pass
    def sauvegarder_db(self): pass
    def charger_db(self): pass
    def calculer_statistiques(self): pass
    def exporter_csv(self): pass
    # ... 50 autres méthodes

# Solution : diviser en plusieurs classes spécialisées
# ✅ Bon
class UtilisateurService: pass  # Gestion utilisateurs
class EmailService: pass        # Emails
class RapportService: pass      # Rapports
class DatabaseService: pass     # Base de données

### Piège 3 : Couplage Fort

**Problème** : Classes trop dépendantes les unes des autres.

In [ ]:
# Couplage fort

# ❌ Couplage fort
class EmailService:
    def envoyer(self, message):
        return f"Email: {message}"

class Utilisateur:
    def __init__(self):
        # Dépendance directe : couplage fort!
        self.email_service = EmailService()
    
    def notifier(self, msg):
        return self.email_service.envoyer(msg)

# Problème : impossible de changer EmailService sans modifier Utilisateur


# ✅ Couplage faible : injection de dépendance
class Utilisateur:
    def __init__(self, notification_service):
        # Service injecté : couplage faible!
        self.notification_service = notification_service
    
    def notifier(self, msg):
        return self.notification_service.envoyer(msg)

# Maintenant on peut facilement changer de service
u1 = Utilisateur(EmailService())
# u2 = Utilisateur(SMSService())  # Facile à changer!

## 10. Mini-Exercices

### Exercice 1 : Modéliser un Système avec Composition

Créez un système de bibliothèque avec :

1. Classe `Livre` avec titre, auteur, ISBN
2. Classe `Bibliotheque` qui contient des livres (agrégation)
3. Classe `Membre` qui peut emprunter des livres
4. Méthodes : `emprunter_livre()`, `retourner_livre()`, `lister_livres()`

Les livres existent indépendamment de la bibliothèque (agrégation, pas composition).

In [ ]:
# Votre code ici


### Exercice 2 : Dessiner Associations UML

Pour ce scénario, identifiez les types d'associations (composition, agrégation, héritage) :

**Système universitaire** :
- Une Université a des Facultés
- Une Faculté a des Professeurs
- Un Professeur enseigne des Cours
- Un Cours a des Étudiants inscrits
- Un Étudiant est une Personne
- Un Professeur est une Personne

Dessinez le diagramme en commentaires ou en Mermaid.

In [ ]:
# Votre diagramme ici (en commentaires ou code Mermaid)


### Exercice 3 : Interface et SRP

Créez un système de paiement avec interfaces :

1. Interface `PaymentProcessor` avec méthode `process_payment(amount)`
2. Implémentations : `CreditCardPayment`, `PayPalPayment`, `BitcoinPayment`
3. Classe `Order` qui utilise un `PaymentProcessor` (composition)
4. Chaque classe doit avoir une seule responsabilité (SRP)

Démontrez le polymorphisme en traitant différents types de paiements.

In [ ]:
# Votre code ici


## Solutions

### Solution Exercice 1

In [ ]:
# Solution complète de l'exercice 1

class Livre:
    """Livre qui peut exister indépendamment."""
    
    def __init__(self, titre, auteur, isbn):
        self.titre = titre
        self.auteur = auteur
        self.isbn = isbn
        self.est_emprunte = False
    
    def __repr__(self):
        statut = "emprunté" if self.est_emprunte else "disponible"
        return f"{self.titre} par {self.auteur} ({statut})"


class Membre:
    """Membre de la bibliothèque."""
    
    def __init__(self, nom, numero_membre):
        self.nom = nom
        self.numero_membre = numero_membre
        self.livres_empruntes = []  # Agrégation
    
    def emprunter(self, livre):
        if livre.est_emprunte:
            return f"❌ {livre.titre} est déjà emprunté"
        livre.est_emprunte = True
        self.livres_empruntes.append(livre)
        return f"✅ {self.nom} a emprunté {livre.titre}"
    
    def retourner(self, livre):
        if livre not in self.livres_empruntes:
            return f"❌ {self.nom} n'a pas emprunté ce livre"
        livre.est_emprunte = False
        self.livres_empruntes.remove(livre)
        return f"✅ {self.nom} a retourné {livre.titre}"


class Bibliotheque:
    """Bibliothèque qui contient des livres (agrégation)."""
    
    def __init__(self, nom):
        self.nom = nom
        self.livres = []  # Agrégation : les livres existent indépendamment
        self.membres = []
    
    def ajouter_livre(self, livre):
        self.livres.append(livre)
        return f"Livre ajouté: {livre.titre}"
    
    def inscrire_membre(self, membre):
        self.membres.append(membre)
        return f"Membre inscrit: {membre.nom}"
    
    def lister_livres(self):
        return "\n".join(str(livre) for livre in self.livres)
    
    def livres_disponibles(self):
        return [livre for livre in self.livres if not livre.est_emprunte]


# Démonstration
print("=== Système de Bibliothèque ===")

# Les livres sont créés indépendamment
livre1 = Livre("Clean Code", "Robert Martin", "978-0132350884")
livre2 = Livre("Python Crash Course", "Eric Matthes", "978-1593279288")
livre3 = Livre("Design Patterns", "Gang of Four", "978-0201633610")

# La bibliothèque regroupe des livres existants (agrégation)
biblio = Bibliotheque("Bibliothèque Centrale")
print(biblio.ajouter_livre(livre1))
print(biblio.ajouter_livre(livre2))
print(biblio.ajouter_livre(livre3))

# Inscription de membres
alice = Membre("Alice Dupont", "M001")
bob = Membre("Bob Martin", "M002")
print(biblio.inscrire_membre(alice))
print(biblio.inscrire_membre(bob))

print("\n=== État initial ===")
print(biblio.lister_livres())

print("\n=== Emprunts ===")
print(alice.emprunter(livre1))
print(bob.emprunter(livre2))
print(alice.emprunter(livre1))  # Déjà emprunté

print("\n=== État après emprunts ===")
print(biblio.lister_livres())

print("\n=== Retours ===")
print(alice.retourner(livre1))

print("\n=== Livres disponibles ===")
for livre in biblio.livres_disponibles():
    print(f"  - {livre}")

# Les livres existent toujours même si on supprime la bibliothèque
del biblio
print(f"\nLe livre existe toujours: {livre1}")

### Solution Exercice 2

In [ ]:
# Solution de l'exercice 2 : Diagramme UML

mermaid_solution = """
classDiagram
    %% Héritage : Etudiant et Professeur sont des Personnes
    Personne <|-- Etudiant
    Personne <|-- Professeur
    
    %% Composition : Université possède des Facultés
    Universite *-- "1..*" Faculte : contient
    
    %% Agrégation : Faculté a des Professeurs
    Faculte o-- "1..*" Professeur : emploie
    
    %% Association : Professeur enseigne des Cours
    Professeur -- "1..*" Cours : enseigne
    
    %% Association : Cours a des Étudiants
    Cours -- "0..*" Etudiant : inscrit
    
    class Personne {
        +String nom
        +String prenom
        +Date dateNaissance
    }
    
    class Etudiant {
        +String numeroEtudiant
        +String formation
    }
    
    class Professeur {
        +String matiere
        +float salaire
    }
    
    class Universite {
        +String nom
    }
    
    class Faculte {
        +String nom
        +String domaine
    }
    
    class Cours {
        +String code
        +String intitule
        +int credits
    }
"""

print("Solution Exercice 2 - Diagramme UML:")
print(mermaid_solution)
print("\n📝 Visualisez sur https://mermaid.live")
print("\nAnalyse des relations:")
print("  1. Université *-- Faculté : COMPOSITION (faculté n'existe pas sans université)")
print("  2. Faculté o-- Professeur : AGREGATION (professeur peut changer de faculté)")
print("  3. Professeur -- Cours : ASSOCIATION (professeur enseigne des cours)")
print("  4. Cours -- Étudiant : ASSOCIATION (étudiants s'inscrivent aux cours)")
print("  5. Personne <|-- Étudiant/Professeur : HERITAGE (is-a)")

### Solution Exercice 3

In [ ]:
# Solution complète de l'exercice 3

from abc import ABC, abstractmethod


# Interface PaymentProcessor
class PaymentProcessor(ABC):
    """Interface : contrat pour tous les processeurs de paiement."""
    
    @abstractmethod
    def process_payment(self, amount: float) -> str:
        """Traite un paiement et retourne un message de confirmation."""
        pass


# Implémentations concrètes
class CreditCardPayment(PaymentProcessor):
    """Responsabilité : Paiement par carte de crédit."""
    
    def __init__(self, card_number, cvv):
        self.card_number = card_number[-4:]  # Derniers 4 chiffres
        self.cvv = cvv
    
    def process_payment(self, amount: float) -> str:
        # Logique de paiement carte
        return f"💳 Paiement de {amount:.2f}€ par carte ****{self.card_number} - Approuvé"


class PayPalPayment(PaymentProcessor):
    """Responsabilité : Paiement par PayPal."""
    
    def __init__(self, email):
        self.email = email
    
    def process_payment(self, amount: float) -> str:
        # Logique de paiement PayPal
        return f"🅿️ Paiement de {amount:.2f}€ via PayPal ({self.email}) - Approuvé"


class BitcoinPayment(PaymentProcessor):
    """Responsabilité : Paiement par Bitcoin."""
    
    def __init__(self, wallet_address):
        self.wallet_address = wallet_address
    
    def process_payment(self, amount: float) -> str:
        # Conversion fictive EUR -> BTC
        btc_amount = amount / 50000  # 1 BTC = 50000€
        return f"₿ Paiement de {btc_amount:.6f} BTC (≈{amount:.2f}€) au wallet {self.wallet_address[:8]}... - Approuvé"


# Classe Order qui utilise l'interface (SRP : gestion de commande uniquement)
class Order:
    """Responsabilité : Gestion d'une commande."""
    
    def __init__(self, order_id, items, total_amount):
        self.order_id = order_id
        self.items = items
        self.total_amount = total_amount
        self.is_paid = False
    
    def checkout(self, payment_processor: PaymentProcessor):
        """Finalise la commande avec un processeur de paiement."""
        if self.is_paid:
            return "❌ Cette commande a déjà été payée"
        
        # Polymorphisme : n'importe quel PaymentProcessor fonctionne
        result = payment_processor.process_payment(self.total_amount)
        self.is_paid = True
        return f"Commande #{self.order_id}\n{result}"
    
    def summary(self):
        status = "Payée ✅" if self.is_paid else "En attente ⏳"
        return f"Commande #{self.order_id} - {len(self.items)} article(s) - {self.total_amount:.2f}€ - {status}"


# Démonstration du polymorphisme
print("=== Système de Paiement Polymorphe ===")

# Création de commandes
order1 = Order("ORD001", ["Laptop", "Mouse"], 1200.00)
order2 = Order("ORD002", ["Book", "Pen"], 35.50)
order3 = Order("ORD003", ["Monitor"], 450.00)

print("\nÉtat initial:")
print(order1.summary())
print(order2.summary())
print(order3.summary())

# Différents moyens de paiement (polymorphisme)
credit_card = CreditCardPayment("1234567890123456", "123")
paypal = PayPalPayment("user@example.com")
bitcoin = BitcoinPayment("1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa")

print("\n=== Paiements ===")
print(order1.checkout(credit_card))  # Paiement par carte
print()
print(order2.checkout(paypal))       # Paiement par PayPal
print()
print(order3.checkout(bitcoin))      # Paiement par Bitcoin

print("\n=== État final ===")
print(order1.summary())
print(order2.summary())
print(order3.summary())

# Démonstration de la flexibilité
print("\n=== Flexibilité ===")
order4 = Order("ORD004", ["Phone"], 899.99)
print("On peut choisir n'importe quel processeur au moment du checkout:")
print(order4.checkout(paypal))  # Facile de changer!

## Récapitulatif

Dans ce notebook, vous avez appris :

✅ Les différents types d'associations entre classes  
✅ La composition : relation forte "a-un" avec dépendance de cycle de vie  
✅ L'agrégation : relation faible "a-un" sans dépendance de cycle de vie  
✅ Le principe "Prefer Composition Over Inheritance"  
✅ Les interfaces : contrats de comportement  
✅ La différence entre classes abstraites et interfaces  
✅ Le principe de responsabilité unique (SRP)  
✅ La notation UML pour les associations (losanges vide/plein)  
✅ Les pièges : trop d'héritage, God Class, couplage fort  

**Prochaine étape :** Découvrir l'UML en détail et les design patterns classiques.